Import South Korea data

In [1]:
import pandas as pd
import glob
import os

#get path for os 
path = os.getcwd() + "/data_source/SK/coronavirusdataset/Time.csv"
path = path.replace("/",os.path.sep)

columns = ["total_cases","daily_cases","actual_cases","total_healed","daily_healed","total_dead","daily_dead","total_test","daily_test"]


dataset = pd.read_csv(path, index_col = 0)
dataset = dataset.fillna(0)

Grouping by date

Creating a new dataframe with our specific format

In [3]:
sk_dataset = pd.DataFrame().reindex_like(dataset)
sk_dataset = pd.DataFrame(index = dataset.index , columns = columns) 
sk_dataset = sk_dataset.fillna(0)

,total_cases,daily_cases,actual_cases,total_healed,daily_healed,total_dead,daily_dead,total_test,daily_test
date,,,,,,,,,
2020-01-20,0,0,0,0,0,0,0,0,0
2020-01-21,0,0,0,0,0,0,0,0,0
2020-01-22,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2020-03-18,0,0,0,0,0,0,0,0,0
2020-03-19,0,0,0,0,0,0,0,0,0
2020-03-20,0,0,0,0,0,0,0,0,0


Filling the new dataframe with the avaiable data

In [4]:
from datetime import datetime, timedelta
day = []
for days in dataset.index:
    try:
        #from 2nd row ---- to subtract a day from the date we had to convert it to a date and then back to a string
        yesterday = (datetime.strptime(days,"%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
        sk_dataset.loc[days]["daily_cases"] = dataset.loc[days]["confirmed"] - dataset.loc[yesterday]["confirmed"]
        sk_dataset.loc[days]["daily_healed"] = dataset.loc[days]["released"] - dataset.loc[yesterday]["released"]
        sk_dataset.loc[days]["daily_dead"] = dataset.loc[days]["deceased"] - dataset.loc[yesterday]["deceased"]
        sk_dataset.loc[days]["daily_test"] = dataset.loc[days]["test"] - dataset.loc[yesterday]["test"]
    except:
        #First row
        sk_dataset.loc[days]["daily_cases"] = dataset.loc[days]["confirmed"]
        sk_dataset.loc[days]["daily_healed"] = dataset.loc[days]["released"] 
        sk_dataset.loc[days]["daily_dead"] = dataset.loc[days]["deceased"]
    finally:
        #Single row
        sk_dataset.loc[days]["total_cases"] = dataset.loc[days]["confirmed"]
        sk_dataset.loc[days]["actual_cases"] = dataset.loc[days]["confirmed"] - dataset.loc[days]["released"] - dataset.loc[days]["deceased"]
        sk_dataset.loc[days]["total_healed"] = dataset.loc[days]["released"]
        sk_dataset.loc[days]["total_dead"] = dataset.loc[days]["deceased"]
        sk_dataset.loc[days]["total_test"] = dataset.loc[days]["test"]


,total_cases,daily_cases,actual_cases,total_healed,daily_healed,total_dead,daily_dead,total_test,daily_test
date,,,,,,,,,
2020-01-20,1,1,1,0,0,0,0,1,0
2020-01-21,1,0,1,0,0,0,0,1,0
2020-01-22,1,0,1,0,0,0,0,4,3
2020-01-23,1,0,1,0,0,0,0,22,18
2020-01-24,2,1,2,0,0,0,0,27,5
...,...,...,...,...,...,...,...,...,...
2020-03-18,8413,93,6789,1540,139,84,3,295647,8931
2020-03-19,8565,152,6527,1947,407,91,7,307024,11377
2020-03-20,8652,87,6325,2233,286,94,3,316664,9640


Converting dates from String to Datetime

In [5]:
import datetime as dt

index_list = sk_dataset.index.tolist()

for i in range(len(index_list)):
    index_list[i] = dt.datetime.strptime(index_list[i], '%Y-%m-%d').date()

sk_dataset.index = index_list
sk_dataset.index.name = 'date'


datetime.date(2020, 1, 20)

In [6]:
#get path for os 
path = os.getcwd() + "/parsed_data/south_korea/"
path = path.replace("/",os.path.sep)
#save data as CSV 
sk_dataset.to_csv (path + "south_korea.csv", index = True, header=True)